In [1]:
import numpy as np
import random
import math
from matplotlib import pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
import time
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
np.random.seed(5)
from sklearn.utils.extmath import row_norms
from sklearn.utils.extmath import row_norms, squared_norm, stable_cumsum
import scipy.sparse as sp
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from scipy.spatial import distance 

Initialize the data points and Random centers
===========================


In [2]:
def generateRandomCenters(c,x):
    centers=[]
    _min= (np.min(x))
    _max= (np.max(x))
    t= np.mean(x,axis=0)    
    for i in range(c):
        centers.append( t + 0.01*i)   
    return np.asarray(centers) 


In [3]:
# intialize random member ship
def initializeMembershipMatrix(n,cluster):
    membership_mat = list()
    for i in range(n.shape[0]):
        random_num_list = [ np.random.rand() for i in range(cluster)]
        summation = sum(random_num_list) 
        temp_list = [x/summation for x in random_num_list]
        membership_mat.append(temp_list) 
    return np.asarray(membership_mat) 

In [4]:
def updateMembershipValue3_2(v,val):
    distances = cdist(val, v,metric='euclidean')
    return distances

In [5]:

def CalculateWeightValues(cluster_center,val,q):
        
    membership=[]    
    distances = cdist(val, v,metric='euclidean')[:,q]  
    sumdistance=np.sum(distances)
    membership= distances/sumdistance 
    
    return membership


In [6]:
def CalculateWeightValuesByIndex(cluster_center,val,q,index):     
    distances = cdist(val, cluster_center,metric='euclidean')[:,q]  
    sumdistance=np.sum(distances) 
    membership= distances[index]/sumdistance        
    return membership


In [7]:
def calcError(center,mem):
    dist=0
    for i in range(mem.shape[0]):
        dist+=np.linalg.norm(center - mem[i])
    return dist

In [8]:
def moveVtoTheCenter(v,u,x):
    for k in range( cluster):
        items=[]
        for i  in range(  (x.shape[0])):
            if(u[i,k]>=np.max(u[i])):
                items.append(x[i])
        v[k]=np.mean(items, axis=0)
    return v

In [9]:
def _k_init(X, n_clusters, x_squared_norms, random_state, n_local_trials=None):
    """Init n_clusters seeds according to k-means++
    Parameters
    ----------
    X : array or sparse matrix, shape (n_samples, n_features)
        The data to pick seeds for. To avoid memory copy, the input data
        should be double precision (dtype=np.float64).
    n_clusters : integer
        The number of seeds to choose
    x_squared_norms : array, shape (n_samples,)
        Squared Euclidean norm of each data point.
    random_state : int, RandomState instance
        The generator used to initialize the centers. Use an int to make the
        randomness deterministic.
        See :term:`Glossary <random_state>`.
    n_local_trials : integer, optional
        The number of seeding trials for each center (except the first),
        of which the one reducing inertia the most is greedily chosen.
        Set to None to make the number of trials depend logarithmically
        on the number of seeds (2+log(k)); this is the default.
    Notes
    -----
    Selects initial cluster centers for k-mean clustering in a smart way
    to speed up convergence. see: Arthur, D. and Vassilvitskii, S.
    "k-means++: the advantages of careful seeding". ACM-SIAM symposium
    on Discrete algorithms. 2007
    Version ported from http://www.stanford.edu/~darthur/kMeansppTest.zip,
    which is the implementation used in the aforementioned paper.
    """
    n_samples, n_features = X.shape

    centers = np.empty((n_clusters, n_features), dtype=X.dtype)

    assert x_squared_norms is not None, 'x_squared_norms None in _k_init'

    # Set the number of local seeding trials if none is given
    if n_local_trials is None:
        # This is what Arthur/Vassilvitskii tried, but did not report
        # specific results for other than mentioning in the conclusion
        # that it helped.
        n_local_trials = 2 + int(np.log(n_clusters))

    # Pick first center randomly
    center_id = random_state.randint(n_samples)
    if sp.issparse(X):
        centers[0] = X[center_id].toarray()
    else:
        centers[0] = X[center_id]

    # Initialize list of closest distances and calculate current potential
    closest_dist_sq = euclidean_distances(
        centers[0, np.newaxis], X, Y_norm_squared=x_squared_norms,
        squared=True)
    current_pot = closest_dist_sq.sum()

    # Pick the remaining n_clusters-1 points
    for c in range(1, n_clusters):
        # Choose center candidates by sampling with probability proportional
        # to the squared distance to the closest existing center
        rand_vals = random_state.random_sample(n_local_trials) * current_pot
        candidate_ids = np.searchsorted(stable_cumsum(closest_dist_sq),
                                        rand_vals)

        # Compute distances to center candidates
        distance_to_candidates = euclidean_distances(
            X[candidate_ids], X, Y_norm_squared=x_squared_norms, squared=True)

        # Decide which candidate is the best
        best_candidate = None
        best_pot = None
        best_dist_sq = None
        for trial in range(n_local_trials):
            # Compute potential when including center candidate
            new_dist_sq = np.minimum(closest_dist_sq,
                                     distance_to_candidates[trial])
            new_pot = new_dist_sq.sum()

            # Store result if it is the best local trial so far
            if (best_candidate is None) or (new_pot < best_pot):
                best_candidate = candidate_ids[trial]
                best_pot = new_pot
                best_dist_sq = new_dist_sq

        # Permanently add best center candidate found in local tries
        if sp.issparse(X):
            centers[c] = X[best_candidate].toarray()
        else:
            centers[c] = X[best_candidate]
        current_pot = best_pot
        closest_dist_sq = best_dist_sq

    return centers

In [10]:
cluster=35
data=pd.read_csv('../../../dataset/a2.txt', header=None, sep=',')  #
datasetname='a2'
x= np.asarray(data.iloc[:, :])  
data.head()  

,0,1
0,53920,42968
1,52019,42206
2,52570,42476
3,54220,42081
4,54268,43420


In [11]:
c = _k_init(x,cluster, row_norms(x, squared=True), np.random.RandomState()) 

In [12]:
error=[]
for i in range(10):
    
    data=shuffle(data)
    x= np.asarray(data.iloc[:, :])    

    v=c 
    w=updateMembershipValue3_2(v,x)
    ny=np.argmin(w,axis=1)

    val=x
    
    for t in range(50):
        for e in range(x.shape[0]):
            q = ny[e]   
            # find the points in the cluster q  
            val=x[ny == q,:] 
            # find the index of that element 
            index= -1
            for r in range(val.shape[0]):
                if(val[r][0]==x[e][0] and val[r][1]==x[e][1]):
                    index=r 

            weight=CalculateWeightValuesByIndex(v,val,q,index)                 
            v[q]= v[q]+ weight*(x[e]- v[q])  

            w=updateMembershipValue3_2(v,x) 
            ny=np.argmin(w,axis=1)      

    centers = v
    labels=ny
    dst=0
    
    for j in range(len(labels)):
        dst+=distance.euclidean(list(data.iloc[j,:]),centers[labels[j]]) 
    error.append(dst)
    
    print(i)

0
1
2
3
4
5
6
7
8
9


In [13]:
print(error)
print('mean ',np.mean(error))
print('std ',np.std(error))

[9195582.90463736, 9195481.864174267, 9196441.185787126, 9193998.293741664, 9195907.541419469, 9194260.977590095, 9194986.050317775, 9200410.236492263, 9190708.606903018, 9198689.306304688]
mean  9195646.696736773
std  2492.7476983251086


In [14]:
error

[9195582.90463736,
 9195481.864174267,
 9196441.185787126,
 9193998.293741664,
 9195907.541419469,
 9194260.977590095,
 9194986.050317775,
 9200410.236492263,
 9190708.606903018,
 9198689.306304688]